Importing the Dependencies

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import difflib
from sklearn.metrics.pairwise import cosine_similarity

Difflib helps to find the closest match to the movie the user will enter

In [ ]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive
!pip install matplotlib-venn
import pydot

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 155629 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.2.2-3.1ubuntu0.7_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.2.2-3.1ubuntu0.7) ...
Setting up libarchive-dev:amd64 (3.2.2-3.1ubuntu0.7) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
     |████████████████████████████████| 154 kB 6.2 MB/s 
  Created wheel for libarchive: filename=libarchive-0.4.7-py3-none-any.whl size=31646 sha256=91737e0275d7e265bc52ca2f95e0d731b0e29f2a2d0a65f6d242913638fe286e
  Stored in directory: /root/.cache/pip/wheels/63/b1/c6/b3da79bec2012175bd43603eed98ef8548ac1733b77c1d4330
Successfully built libarchive


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

Textual data to numerical using TfidfVectorizer

Data Collection and Pre-Processing

In [ ]:
from google.colab import files
upload = files.upload()

Saving movies.csv to movies.csv


In [ ]:
movies_data = pd.read_csv('movies.csv')

In [ ]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
for col in movies_data.columns:
  print(col)

index
budget
genres
homepage
id
keywords
original_language
original_title
overview
popularity
production_companies
production_countries
release_date
revenue
runtime
spoken_languages
status
tagline
title
vote_average
vote_count
cast
crew
director


In [ ]:
movies_data.shape

(4803, 24)

(Rows, Columns)

In [ ]:
selected_features = ['keywords', 'tagline', 'director','cast','genres']
print(selected_features)

['keywords', 'tagline', 'director', 'cast', 'genres']


Replacing null values with null string because textual data consists null values.

In [ ]:
for feat in selected_features:
  movies_data[feat] = movies_data[feat].fillna('')

In [ ]:
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['cast']+' '+movies_data['director']+' '+movies_data['tagline']

In [ ]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  Edward Burns Kerry Bish\u00e9 ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      Daniel Henney Eliza Coupe Bill Paxton Alan R...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


Convert text data to feature vectors for cosine similarity. As Cosine similarity works effeciently on numerical values

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
feature_vectors = vectorizer.fit_transform(combined_features)
print(feature_vectors)

  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

Getting similarity scores using cosine similarity

In [ ]:
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


Comparing Avatar with all other movies and giving the similarity score.

In [ ]:
print(similarity.shape)

(4803, 4803)


Getting he movie name from the user

In [ ]:
movie_name = input("Enter the movie name: ")

Enter the movie name: The Dark Knight


Creating a list with all movies given in dataset

In [ ]:
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

Finding the close match for the movie name by the user

In [ ]:
find_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_match)

['The Dark Knight', 'The Dark Knight Rises', 'Shark Night']


In [ ]:
close_match = find_match[0]
print(close_match)

The Dark Knight


Find the index of the movie

In [ ]:
movie_index = movies_data[movies_data.title == close_match]['index'].values[0]
print(movie_index)

65


In [ ]:
similarity_score = list(enumerate(similarity[movie_index]))
similarity_score

[(0, 0.006299544406410075),
 (1, 0.005498255350089556),
 (2, 0.0473001764689897),
 (3, 0.4739064762535151),
 (4, 0.00613416569239541),
 (5, 0.036658801580734084),
 (6, 0.0),
 (7, 0.00635857299084362),
 (8, 0.01131280943223584),
 (9, 0.09095298891343916),
 (10, 0.10373740062549588),
 (11, 0.06576097647103317),
 (12, 0.006432156504715376),
 (13, 0.005807343771874207),
 (14, 0.10251041749018085),
 (15, 0.0),
 (16, 0.006029278836837788),
 (17, 0.00615310441324767),
 (18, 0.016483322527117667),
 (19, 0.005462958379896951),
 (20, 0.0062733985777304714),
 (21, 0.005286862491965671),
 (22, 0.0),
 (23, 0.0),
 (24, 0.008995116876377645),
 (25, 0.009174262745461691),
 (26, 0.006562600308185515),
 (27, 0.01109200729242231),
 (28, 0.012177715042626155),
 (29, 0.035672367679855044),
 (30, 0.0356653940170164),
 (31, 0.006190862932664915),
 (32, 0.0),
 (33, 0.012214035705777894),
 (34, 0.0),
 (35, 0.018166844283804266),
 (36, 0.01636977644681597),
 (37, 0.0),
 (38, 0.006429149625438347),
 (39, 0.05396

In [ ]:
len(similarity_score)

4803

In [ ]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
sorted_similar_movies

[(65, 1.0000000000000002),
 (3, 0.4739064762535151),
 (119, 0.4404491174122588),
 (1196, 0.2532971333588242),
 (428, 0.20292112115415228),
 (210, 0.1877180995254057),
 (1740, 0.1824535837041526),
 (1720, 0.179301228077641),
 (1359, 0.1775581506611392),
 (813, 0.16160582880593957),
 (2717, 0.1530196833800321),
 (1658, 0.14542524672611318),
 (739, 0.129544576684201),
 (163, 0.12903278496345233),
 (72, 0.12823188712461162),
 (1449, 0.12314732325321902),
 (95, 0.12007985540794443),
 (1767, 0.11874932692335252),
 (1087, 0.11637279215043445),
 (157, 0.11633448979773872),
 (1143, 0.11557736285159624),
 (140, 0.11543798495606843),
 (1035, 0.11426465179181247),
 (299, 0.11325756225692478),
 (1033, 0.11308240690582054),
 (806, 0.11263270725610099),
 (2793, 0.11250519758961225),
 (1296, 0.11214505868657283),
 (870, 0.1113134523460697),
 (303, 0.11120783664340315),
 (472, 0.1105184774049977),
 (3706, 0.11016038055413406),
 (1420, 0.10918038300103153),
 (3492, 0.10889154923089568),
 (246, 0.1045245

Print the name of similar movies

In [ ]:
print("Movies suggested for you: ")
i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index == index]['title'].values[0]
  if (i<30):
    print(i, '.', title_from_index)
    i+=1

Movies suggested for you: 
1 . The Dark Knight
2 . The Dark Knight Rises
3 . Batman Begins
4 . The Prestige
5 . Batman Returns
6 . Batman & Robin
7 . Kick-Ass 2
8 . Kick-Ass
9 . Batman
10 . Superman
11 . Brokeback Mountain
12 . The Imaginarium of Doctor Parnassus
13 . London Has Fallen
14 . Watchmen
15 . Suicide Squad
16 . The Order
17 . Interstellar
18 . Suspect Zero
19 . The Pledge
20 . Exodus: Gods and Kings
21 . A Knight's Tale
22 . White House Down
23 . Jonah Hex
24 . Batman Forever
25 . Insomnia
26 . Captain Corelli's Mandolin
27 . The Killer Inside Me
28 . Superman III
29 . Superman II


In [ ]:
movie_name = input("Enter the movie name: ")

list_of_all_titles = movies_data['title'].tolist()

find_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_match[0]

movie_index = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[movie_index]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print("Movies suggested for you: ")
i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index == index]['title'].values[0]
  if (i<30):
    print(i, '.', title_from_index)
    i+=1

Enter the movie name: Casino Royale
Movies suggested for you: 
1 . Casino Royale
2 . Star Wars: Episode I - The Phantom Menace
3 . Pan
4 . Skyfall
5 . The Three Musketeers
6 . Tea with Mussolini
7 . Quantum of Solace
8 . The Mask of Zorro
9 . The Matrix Revolutions
10 . Underworld: Rise of the Lycans
11 . Flame & Citron
12 . House Party 2
13 . I Dreamed of Africa
14 . The Second Best Exotic Marigold Hotel
15 . The Golden Compass
16 . The Hunt
17 . Harry Brown
18 . Dracula Untold
19 . Enough
20 . Once Upon a Time in Mexico
21 . Goal!: The Dream Begins
22 . A Farewell to Arms
23 . My Week with Marilyn
24 . Cadillac Records
25 . Mrs Henderson Presents
26 . Trippin'
27 . The Adventures of Pluto Nash
28 . Nine
29 . Home on the Range
